In [4]:
import dspy
from dspy.retrieve.weaviate_rm import WeaviateRM
import weaviate

weaviate_client = weaviate.connect_to_local()

#command_r_plus = dspy.Cohere(model="command-r-plus", max_tokens=4000, api_key=cohere_api_key)
#command_r = dspy.Cohere(model="command-r", max_tokens=4000, api_key=cohere_api_key)
#gpt4 = dspy.OpenAI(model="gpt-4-1106-preview", max_tokens=4000, model_type="chat")
#gpt_turbo = dspy.OpenAI(model="gpt-3.5-turbo", max_tokens=4000, model_type="chat")
mistral_ollama = dspy.OllamaLocal(model="mistral", max_tokens=4000, timeout_s=480)
llama2_ollama = dspy.OllamaLocal(model="llama2", max_tokens=4000, timeout_s=480)


lms = [{"name": "Mistral-7B", "lm": mistral_ollama},
       {"name": "Llama2-7B", "lm": llama2_ollama}]

retriever_model = WeaviateRM("WeaviateBlogChunk", weaviate_client=weaviate_client)
dspy.settings.configure(lm=mistral_ollama, rm=retriever_model)

In [13]:
from pydantic import BaseModel, validator

class UpdatedPropertyValue(BaseModel):
    property_value: str

    @field_validator('property_value')
    def check_starts_with(cls, v):
        invalid_prefixes = ["Property Name", "References", "Instruction", "Property Value"]
        for prefix in invalid_prefixes:
            if v.startswith(prefix):
                raise ValueError(f"property_value cannot start with '{prefix}'")
        return v

/var/folders/41/8dp_379x15d8zz4ppsjthdw40000gn/T/ipykernel_87328/412093697.py:6: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  @validator('property_value')


In [19]:
class UpdateProperty(dspy.Signature):
    """Your task is to generate the value of a property by following the instruction using the provided name-value property references."""

    property_name: str = dspy.InputField(desc="The name of the property that you should update.")
    references: str = dspy.InputField(
        desc="The name-value property pairs that you should refer to while updating the property."
    )
    instruction: str = dspy.InputField(
        desc="The prompt to use when generating the content of the updated property value."
    )
    property_value: UpdatedPropertyValue = dspy.OutputField(desc="The value of the updated property as a string.")
        
class Program(dspy.Module):
    def __init__(self) -> None:
        self.predict = dspy.TypedPredictor(UpdateProperty)

    def forward(self, property_name: str, references: str, instruction: str) -> str:
        prediction: dspy.Prediction = self.predict(
            property_name=property_name, references=references, instruction=instruction
        )
        return prediction.property_value.property_value

In [20]:
summarize_test = {
    "property_name": "summary",
    "references": """
    title-LeBron James,
    content-LeBron Raymone James Sr. (/ləˈbrɒn/ lə-BRON; born December 30, 1984) is an American professional basketball player for the Los Angeles Lakers of the National Basketball Association (NBA). Nicknamed "King James", he is widely recognized as one of the greatest players in the history of the sport and is often compared to Michael Jordan in debates over the greatest basketball player of all time.[a] He has competed in 10 NBA Finals (with eight consecutive appearances from 2011 to 2018), winning four NBA championships.[1] He also won the inaugural NBA Cup in 2023 with the Lakers, and two Olympic gold medals as a member of the U.S. men's national team.
    """,
    "instruction": "Please summarize the content of these references."
}

for lm_dict in lms:
    lm, name = lm_dict["lm"], lm_dict["name"]
    with dspy.context(lm=lm):
        print(f"\033[91mResult for {name}\n")
        print(f"\033[0m{Program()(**summarize_test)} \n")

Result for Mistral-7B

LeBron Raymone James is a four-time NBA champion basketball player for the Los Angeles Lakers. He is widely regarded as one of the greatest players in NBA history and has competed in 10 NBA Finals, winning a total of 5 championships. James also won Olympic gold medals as a member of the U.S. men's national team. 

Result for Llama2-7B

LeBron James is widely recognized as one of the greatest basketball players of all time, with numerous accolades and achievements throughout his career. He has competed in 10 NBA Finals, winning four championships, and also won two Olympic gold medals as a member of the U.S. men's national team. A summary of his references is: LeBron Raymone James Sr. (/ləˈbrɒn/ lə-BRON; born December 30, 1984) is an American professional basketball player for the Los Angeles Lakers of the National Basketball Association (NBA). 



In [21]:
birthday_test = {
    "property_name": "birthday",
    "references": """
    title-LeBron James,
    content-LeBron Raymone James Sr. (/ləˈbrɒn/ lə-BRON; born December 30, 1984) is an American professional basketball player for the Los Angeles Lakers of the National Basketball Association (NBA). Nicknamed "King James", he is widely recognized as one of the greatest players in the history of the sport and is often compared to Michael Jordan in debates over the greatest basketball player of all time.[a] He has competed in 10 NBA Finals (with eight consecutive appearances from 2011 to 2018), winning four NBA championships.[1] He also won the inaugural NBA Cup in 2023 with the Lakers, and two Olympic gold medals as a member of the U.S. men's national team.
    """,
    "instruction": "Please extract the value for `birthday` from the `references`."
}

for lm_dict in lms:
    lm, name = lm_dict["lm"], lm_dict["name"]
    with dspy.context(lm=lm):
        print(f"\033[91mResult for {name}\n")
        print(f"\033[0m{Program()(**birthday_test)} \n")

Result for Mistral-7B

December 30, 1984 

Result for Llama2-7B

December 30, 1984 



In [22]:
crm_test = {
    "property_name": "generative_feedback_loop_use_case",
    "references": """
    title-LeBron James,
    content-LeBron Raymone James Sr. (/ləˈbrɒn/ lə-BRON; born December 30, 1984) is an American professional basketball player for the Los Angeles Lakers of the National Basketball Association (NBA). Nicknamed "King James", he is widely recognized as one of the greatest players in the history of the sport and is often compared to Michael Jordan in debates over the greatest basketball player of all time.[a] He has competed in 10 NBA Finals (with eight consecutive appearances from 2011 to 2018), winning four NBA championships.[1] He also won the inaugural NBA Cup in 2023 with the Lakers, and two Olympic gold medals as a member of the U.S. men's national team.
    """,
    "instruction": """
    Would this customer be interested in our new Generative Feedback Loop product?
    Generative Feedback Loops describe using AI systems to process data to create new insights or fill in missing values of existing data.
    Please write a short description of how this customer could utilize Generative Feedback Loops."""
}

for lm_dict in lms:
    lm, name = lm_dict["lm"], lm_dict["name"]
    with dspy.context(lm=lm):
        print(f"\033[91mResult for {name}\n")
        print(f"\033[0m{Program()(**crm_test)} \n")

Result for Mistral-7B

Given that LeBron James is a renowned basketball player and a data point in various sports analytics, he could potentially benefit from our new Generative Feedback Loop product. With his extensive career statistics and achievements, this data could be used to train AI systems to generate insights about his performance trends or even predict future outcomes based on historical data. For instance, the system could identify patterns in LeBron's shooting percentages during specific game situations or provide suggestions for optimal team strategies based on past successes. By utilizing Generative Feedback Loops, LeBron and other athletes, coaches, and sports analysts could gain valuable insights to improve performance and make data-driven decisions. 

Result for Llama2-7B

LeBron James, a professional basketball player for the Los Angeles Lakers, could potentially benefit from using Generative Feedback Loops. By leveraging AI systems to process data and create new ins

In [24]:
mistral_ollama.inspect_history(n=1)




Your task is to generate the value of a property by following the instruction using the provided name-value property references.

---

Follow the following format.

Property Name: The name of the property that you should update.

References: The name-value property pairs that you should refer to while updating the property.

Instruction: The prompt to use when generating the content of the updated property value.

Property Value: The value of the updated property as a string.. Respond with a single JSON object. JSON Schema: {"properties": {"property_value": {"title": "Property Value", "type": "string"}}, "required": ["property_value"], "title": "UpdatedPropertyValue", "type": "object"}

---

Property Name: generative_feedback_loop_use_case

References:

    title-LeBron James,
    content-LeBron Raymone James Sr. (/ləˈbrɒn/ lə-BRON; born December 30, 1984) is an American professional basketball player for the Los Angeles Lakers of the National Basketball Association (NBA). Nicknamed

'\n\n\nYour task is to generate the value of a property by following the instruction using the provided name-value property references.\n\n---\n\nFollow the following format.\n\nProperty Name: The name of the property that you should update.\n\nReferences: The name-value property pairs that you should refer to while updating the property.\n\nInstruction: The prompt to use when generating the content of the updated property value.\n\nProperty Value: The value of the updated property as a string.. Respond with a single JSON object. JSON Schema: {"properties": {"property_value": {"title": "Property Value", "type": "string"}}, "required": ["property_value"], "title": "UpdatedPropertyValue", "type": "object"}\n\n---\n\nProperty Name: generative_feedback_loop_use_case\n\nReferences:\n\n    title-LeBron James,\n    content-LeBron Raymone James Sr. (/ləˈbrɒn/ lə-BRON; born December 30, 1984) is an American professional basketball player for the Los Angeles Lakers of the National Basketball Ass